In [ ]:
import urllib, requests
from bs4 import BeautifulSoup
from bs4 import NavigableString, Declaration, Comment
from urllib.request import urlopen
from requests import ConnectionError
from requests.exceptions import ContentDecodingError
from urllib.error import HTTPError, URLError
import csv, re, time, http.client, chardet, sys
from ssl import SSLError

#不要なタグの定義など
def getNavigableStrings(soup):
    if isinstance(soup, NavigableString):
        if type(soup) not in (Comment, Declaration) and soup.strip():
            yield soup
    elif soup.name not in ('script', 'style'):
        for c in soup.contents:
            for g in getNavigableStrings(c):
                yield g

In [ ]:
remove_condition = '.png|.jpg|.jpeg|.pdf'
f = open('link_relational_data.csv', 'r')
dataReader = csv.reader(f)

In [ ]:
#結果の出力用のデータフレームを作る。
data01 =[]
data02 =[]
data03 =[]
data04 =[]
data05 =[]
data06 =[]
data07 =[]
data08 =[]

p = re.compile(r"<[^>]*?>")

for parts in dataReader:
    for site_url in parts:
            time.sleep(1.0) #sleep(秒指定)
            print(site_url)
            try:
                        r = requests.get(site_url,allow_redirects=True)
                        if r.status_code == 200:
                            
                            soup = BeautifulSoup(r.content)
                            #soup = BeautifulSoup(r.text)
                            soup = BeautifulSoup(str(soup).replace("\r", " "))
                            text = ' '.join(getNavigableStrings(soup))
                            text = p.sub("", text)
                            text = text.replace(",","")
                            text = text.replace("\t","")
                            text = text[0:10000]
                                            
                            for link in soup.findAll("a"):
                                try:
                                    if re.match(site_url,link.get("href")):
                                        url = link.get("href")
                                    if not re.match("http",link.get("href")):
                                        if re.match("^/",link.get("href")):
                                            url = site_url + link.get("href")
                                        if not re.match("^/",link.get("href")):
                                            url = site_url + "/" + link.get("href")
                                      
                                    if not re.search(remove_condition,url):
                                        #URL先を取得する
                                        time.sleep(3.0) #sleep(秒指定)
                                        url = url.replace("/./","/")
                                        r2 = requests.get(url,allow_redirects=True)
                                        soup2 = BeautifulSoup(r2.content, 'html.parser')
                                        soup2 = BeautifulSoup(str(soup2).replace("\r", " "))
                                        text2 = ' '.join(getNavigableStrings(soup2))
                                        text2 = p.sub("", text2)
                                        text2 = text2.replace(",","")                                        
                                        text2 = text2.replace("\t","") 
                                        text2 = text2[0:10000]
                                    
                                        #データの格納
                                        data01.append(site_url)
                                        data02.append(text)
                                        data03.append(''.join(link.findAll(text=True)))                                    
                                        data04.append(url)
                                        data05.append(text2)
                                        data06.append(r.status_code)
                                        data07.append(r2.status_code)                                    
                                        data08.append("linklist")
                                        data = zip(data01,data02,data03,data04,data05,data06,data07,data08)
                                        #CSV出力
                                        with open('Company_site_Text_result.csv','wt',errors='backslashreplace') as fout:
                                            writecsv = csv.writer(fout,lineterminator='\n')
                                            writecsv.writerows(data)                                   
                                
                                except TypeError:
                                    a = []
                            
            except HTTPError as e:
                print(e.code)
                
            except requests.exceptions.Timeout:
                print("Timeout occurred")
                
            except ConnectionError:
                print("ConnectionError")
                    
            except SSLError:
                print("SSLError")
                    
            except TypeError:
                print("TypeError")
                    
            except URLError:
                print("URLError")
                    
            except  ValueError:
                print("ValueError")
                    
            except socket.timeout:
                print("Timeout occurred")
                    
            except KeyError as instance:
                print(instance)
                    
            except ConnectionResetError:
                print("ConnectionResetError")
                    
            except http.client.IncompleteRead:
                print("IncompleteRead")
                                        
            except http.client.HTTPException:
                print("HTTPException")
                
            except NotImplementedError:
                print("NotImplementedError")
                
            except requests.exceptions.TooManyRedirects:
                print("TooManyRedirects")
                
            except RecursionError:
                print("RecursionError")
                
            except ContentDecodingError:
                print("ContentDecodingError")

In [ ]:
#%matplotlib inline
import pandas as pd
import numpy as np
import scipy as sp
import sklearn, sys, codecs
sys.setrecursionlimit(10000)
pd.options.display.max_colwidth = 100

#保存したデータの読み込み
df = pd.read_csv(open('Company_site_Text_result.csv','r'), encoding='utf-8', engine='c',header=None) #CSVでデータを読み込む
df.columns = ['url', 'text','anchor_text','link','target_text','status_code','target_status_code','partition'] #列名を付ける。
df.tail(10) #10件表示する。

In [ ]:
#対象の絞込
keyword1 = r'お問い合わせ|お問合せ'
keyword2 = r''
keyword3 = r'\@|＠'

df_filtered = df[df['target_text'].str.contains(keyword1,na=False)]
df_filtered = df_filtered[df_filtered['target_text'].str.contains(keyword2,na=False)]
df_filtered = df_filtered[df_filtered['target_text'].str.contains(keyword3,na=False)]

df_filtered

In [ ]:
#Emailアドレスを正規表現で抽出
df_filtered['email'] = df_filtered['target_text'].str.extract('([\w\.-]+@[\w\.-]+)')
email_extract_company = df_filtered[df_filtered['email'].notnull()]
email_extract_company = email_extract_company[['url','link','email']]
email_extract_company = email_extract_company.drop_duplicates(['url','email'])
email_extract_company.head(10)

In [ ]:
#取得したホームページに占めるメールアドレスらしきホームページの割合（企業ユニーク）
print(str(round(len(email_extract_company.url.value_counts())/len(df.url.value_counts())*100,2)) + "％")

In [ ]:
#CSV出力
email_extract_company.to_csv("email_extract_list.csv",index=None)